Import trained models and test them.

# Imports

In [ ]:
import os
import pickle
import sys
import importlib

import mlflow
import torch
import torch.nn as nn
from torch import nn, Tensor
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('cuda available')
else:
    device = torch.device('cpu')
    print('cuda not available')

# Import model

In [ ]:
run_ids = ['9137e2b725024cb78e0fe4539220153a',
           '2ad9e9608635496ca44f7e28871362c0',
           '972e508debbd44b58acaf891562fcfde',
           'f97e115a81224afda639999106868c0f',
           '8a60045c8d5a4335a2a12c868c1cf0d4',
           '3cfd69d12c974b14a3b60bf5ebe73cf7']  # better bigger dataset, fixed upscaling
# various combinations of symmetric coupling

In [ ]:

client = mlflow.tracking.MlflowClient()

# Import data

In [ ]:
data_folder = r'path/to/data/folder'  # TODO: set this to your local data folder

In [ ]:
rng = np.random.default_rng(42)

# %%
dataset = 'AllenCahn_data_periodic_2000_2.pkl'
data_path = os.path.join(data_folder, dataset)
with open(data_path, 'rb') as f:
    data = pickle.load(f)

N_samples = len(data['solutions'])
inds = rng.choice(N_samples, size=3*N_samples//4, replace=False)
bools = np.zeros(N_samples, dtype=bool)
bools[inds] = True
y_train = data['solutions'][bools]
y_test = data['solutions'][~bools]

x = torch.stack((data['epsilon'], data['mu']), axis=1)
x_train = x[bools]
x_test = x[~bools]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


# Calculate residual before subsampling

In [ ]:
dt = 0.1
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=64, shuffle=False)

from scipy.sparse import diags
L = 1.0
N = y_train.shape[2]  # Number of spatial points

dx = L / N        # Spatial step size
x = np.linspace(0, L, N, endpoint=False)
dt = 0.1
T = y_train.shape[1]  # Number of time steps

# Construct Laplacian with periodic boundary conditions
main_diag = -2.0 * np.ones(N)
off_diag = np.ones(N - 1)
laplacian = diags([off_diag, main_diag, off_diag], offsets=[-1, 0, 1], shape=(N, N)).toarray()
laplacian[0, -1] = laplacian[-1, 0] = 1.0  # periodic BC
laplacian = laplacian / dx**2


In [ ]:
results = {}
residual_real = []
with torch.no_grad():
    # set random seed
    torch.manual_seed(42)

    for j, [epsmu, target] in enumerate(test_loader):
        # print(f'Batch {j}')

        cond = epsmu.to(device)

        target = target.numpy()

        # Allen-Cahn equation: du/dt = eps^2 * laplacian u - (u^3 - mu*u)

        eps = cond[:, 0].reshape(-1, 1, 1).cpu().numpy()
        mu = cond[:, 1].reshape(-1, 1, 1).cpu().numpy()

        lhs2 = (target[:, 1:] - target[:, :-1])/dt
        rhs2 = eps**2 * np.einsum('ij,kmj->kmi', laplacian, target[:, 1:]) - (target[:, :-1]**3 - mu * target[:, :-1])
        diff2 = lhs2 - rhs2  # shape [B, T-1, N]
        residual_real.extend(np.linalg.norm(diff2, axis=(1,2)).tolist())

        ind = 2
        if j == 1:
            plt.imshow(lhs2[ind].T, aspect='auto', cmap='coolwarm')
            plt.colorbar()
            plt.title(f'LHS du/dt, eps={eps[ind][0][0]:3f}, mu={mu[ind][0][0]:3f}')
            plt.show()

            plt.imshow(rhs2[ind].T, aspect='auto', cmap='coolwarm')
            plt.colorbar()
            plt.title(f'RHS du/dt, eps={eps[ind][0][0]:3f}, mu={mu[ind][0][0]:3f}')
            plt.show()

            plt.imshow(target[ind].T, aspect='auto', cmap='coolwarm')
            plt.colorbar()
            plt.title(f'Target, eps={eps[ind][0][0]:3f}, mu={mu[ind][0][0]:3f}')
            plt.show()

            # break  # only first batch for testing


print('Residual (Real):', np.mean(residual_real))

# Subsample

In [ ]:
subsample = 10

if subsample is not None:
    y_train = y_train[:, ::subsample]
    y_test = y_test[:, ::subsample]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=16, shuffle=False)


In [ ]:
target_mean = torch.mean(y_train, dim=(0, 2))
target_std = torch.std(y_train, dim=(0, 2))
print('target_mean:', target_mean[::10])
print('target_std:', target_std[::10])

target_std = target_std.reshape(1, 1, -1, 1)

# Calculate residual after subsampling

In [ ]:
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=64, shuffle=False)

from scipy.sparse import diags
L = 1.0
N = y_train.shape[2]  # Number of spatial points

dx = L / N        # Spatial step size
x = np.linspace(0, L, N, endpoint=False)
dt = 1.0
T = y_train.shape[1]  # Number of time steps

# Construct Laplacian with periodic boundary conditions
main_diag = -2.0 * np.ones(N)
off_diag = np.ones(N - 1)
laplacian = diags([off_diag, main_diag, off_diag], offsets=[-1, 0, 1], shape=(N, N)).toarray()
laplacian[0, -1] = laplacian[-1, 0] = 1.0  # periodic BC
laplacian = laplacian / dx**2


In [ ]:
results = {}
residual_real = []
with torch.no_grad():
    # set random seed
    torch.manual_seed(42)

    for j, [epsmu, target] in enumerate(test_loader):
        # print(f'Batch {j}')

        cond = epsmu.to(device)

        target = target.numpy()

        # Allen-Cahn equation: du/dt = eps^2 * laplacian u - (u^3 - mu*u)

        eps = cond[:, 0].reshape(-1, 1, 1).cpu().numpy()
        mu = cond[:, 1].reshape(-1, 1, 1).cpu().numpy()

        lhs2 = (target[:, 1:] - target[:, :-1])/dt
        rhs2 = eps**2 * np.einsum('ij,kmj->kmi', laplacian, target[:, 1:]) - (target[:, :-1]**3 - mu * target[:, :-1])
        diff2 = lhs2 - rhs2  # shape [B, T-1, N]
        residual_real.extend(np.linalg.norm(diff2, axis=(1,2)).tolist())

        ind = 2
        if j == 1:
            plt.imshow(lhs2[ind].T, aspect='auto', cmap='coolwarm')
            plt.colorbar()
            plt.title(f'LHS du/dt, eps={eps[ind][0][0]:3f}, mu={mu[ind][0][0]:3f}')
            plt.show()

            plt.imshow(rhs2[ind].T, aspect='auto', cmap='coolwarm')
            plt.colorbar()
            plt.title(f'RHS du/dt, eps={eps[ind][0][0]:3f}, mu={mu[ind][0][0]:3f}')
            plt.show()

            plt.imshow(target[ind].T, aspect='auto', cmap='coolwarm')
            plt.colorbar()
            plt.title(f'Target, eps={eps[ind][0][0]:3f}, mu={mu[ind][0][0]:3f}')
            plt.show()

            # break  # only first batch for testing


print('Residual (Real):', np.mean(residual_real))

# Define UNet

In [ ]:
class UNet2D_halfperiodic(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=32, num_layers=4, kernel_size=3, resize_factor=2):
        super(UNet2D_halfperiodic, self).__init__()
        self.depth = num_layers

        if isinstance(resize_factor, int):
            self.resize_factor = [resize_factor]*num_layers
        else:
            assert len(resize_factor) == num_layers, "resize_factor must be an int or a list of length num_layers"
            self.resize_factor = resize_factor

        # Cheatsheet:
        # Input size = [batch, in_channels, height=time steps, width=space points]

        # Encoder path
        self.encoders = nn.ModuleList()
        self.pools = nn.ModuleList()
        prev_channels = in_channels
        for i in range(num_layers):
            self.encoders.append(
                nn.Sequential(
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
                    nn.Conv2d(prev_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
                    nn.ReLU(),
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
                    nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
                    nn.ReLU(),
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
                    nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
                    nn.ReLU(),
                )
            )
            self.pools.append(nn.MaxPool2d(kernel_size=self.resize_factor[i], stride=self.resize_factor[i]))
            prev_channels = hidden_channels

        self.middle_conv = nn.Sequential(
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
            nn.ReLU(),
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
            nn.ReLU(),
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
            nn.ReLU(),
        )

        # Decoder path
        self.decoders = nn.ModuleList()
        for i in range(num_layers):
            # After concatenation, channels = hidden_channels * 2 except at final
            in_ch = hidden_channels * 2  # if i < num_layers - 1 else hidden_channels
            self.decoders.append(
                nn.Sequential(
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
                    nn.Conv2d(in_ch, hidden_channels, kernel_size=kernel_size, padding='valid'),
                    nn.ReLU(),
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
                    nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
                    nn.ReLU(),
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
                    nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
                    nn.ReLU(),
                )
            )

        self.extra_conv = nn.Sequential(
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
            nn.ReLU(),
                    nn.CircularPad2d((1, 1, 0, 0)),
                    nn.ConstantPad2d((0, 0, 1, 1), 0.0),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=kernel_size, padding='valid'),
            nn.ReLU(),
        )

        # Final mapping
        self.final_convs = nn.Sequential(
            nn.Conv2d(hidden_channels + in_channels, hidden_channels, kernel_size=1, padding='valid'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=1, padding='valid'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=1, padding='valid'),
        )

    def forward(self, tau, x, verbose=False):
        # Encoder
        x_in = x.clone()
        skips = []
        if verbose:
            print(f'{"Input shape         :":<30} {x.shape}')
        for encoder, pool in zip(self.encoders, self.pools):
            x = encoder(x)
            skips.append(x)
            if verbose:
                print(f'{"Skip shape          :":<30} {x.shape}')

            x = pool(x)

            if verbose:
                print(f'{"Encoder output shape:":<30} {x.shape}')

        x = self.middle_conv(x)

        if verbose:
            print(f'{"Middle conv shape   :":<30} {x.shape}')

        # Bottleneck output
        # Decoder
        for i in range(0, len(self.decoders)):
            # for periodic interpolation, first add periodic padding on both sides, then interpolate to needed size + 4, then remove extra element
            x = F.pad(x, (1, 1, 0, 0), mode='circular')
            target_size = [skips[-1].shape[2], skips[-1].shape[3]+4]
            x = F.interpolate(x, size=target_size, mode='bilinear')[..., 2:-2]
            if verbose:
                print(f'{"Decoder interpolated shape :":<30} {x.shape}')

            # Skip connection: concatenate with corresponding encoder output
            skip = skips.pop()

            x = torch.cat([x, skip], dim=1)  # if i < len(self.decoders) - 1 else x
            x = self.decoders[i](x)
            if verbose:
                print(f'{"Decoder output shape :":<30} {x.shape}')

        x = self.extra_conv(x)
        x = torch.cat((x, x_in), dim=1)  # concatenate with input for final mapping
        x = self.final_convs(x)  # final mapping to output channels
        return x

# Define Flow

In [ ]:
class Flow(nn.Module):
    def __init__(self):
        super().__init__()
        super(Flow, self).__init__()
        self.Unet = UNet2D_halfperiodic(
            in_channels=4,
            out_channels=1,
        )

    def forward(self, t, x_t, cond) -> Tensor:
        # t.shape = [batch size, 1]
        # x_t.shape = [batch size, 1, time steps, # points]
        # cond.shape = [batch size, 2]

        # print('forward: t.shape', t.shape)
        # print('forward: x_t.shape', x_t.shape)
        # print('forward: cond.shape', cond.shape)

        inp = torch.cat((
            x_t.clone(),
            cond.reshape(-1,2,1,1).expand(-1, 2, x_t.shape[2], x_t.shape[3]),
            t.reshape(-1,1,1,1).expand(-1, 1, x_t.shape[2], x_t.shape[3])
        ), axis=1)

        # print('forward: inp.shape', inp.shape)

        return self.Unet(t, inp, verbose=False)

    def step(self, x_t: Tensor, t_start: Tensor, t_end: Tensor, cond: Tensor) -> Tensor:
        # t: float, current time
        # t_end: float, end time
        # x_t: shape [batch size, 1, time steps, # points], current interpolation
        # cond: shape [batch size, 2], conditioning in the form of mu and epsilon
        t_start = t_start.view(1, 1, 1, 1).expand(x_t.shape[0], 1, 1, 1)
        t_end = t_end.view(1, 1, 1, 1).expand(x_t.shape[0], 1, 1, 1)

        # print('step: x_t.shape', x_t.shape)
        # print('step: t_start.shape', t_start.shape)
        # print('step: t_end.shape', t_end.shape)
        # print('step: cond.shape', cond.shape)

        return (x_t + (t_end - t_start)
                * self(
                    t=t_start + (t_end - t_start) / 2,
                    x_t= x_t + self(x_t=x_t, t=t_start, cond=cond) * (t_end - t_start) / 2,
                    cond=cond
                       )
                )


# Test Flow models

In [ ]:
y_train.shape

In [ ]:
from scipy.sparse import diags
L = 1.0
N = y_train.shape[2]  # Number of spatial points

dx = L / N        # Spatial step size
x = np.linspace(0, L, N, endpoint=False)
dt = 1.0
T = y_train.shape[1]  # Number of time steps

# Construct Laplacian with periodic boundary conditions
main_diag = -2.0 * np.ones(N)
off_diag = np.ones(N - 1)
laplacian = diags([off_diag, main_diag, off_diag], offsets=[-1, 0, 1], shape=(N, N)).toarray()
laplacian[0, -1] = laplacian[-1, 0] = 1.0  # periodic BC
laplacian = laplacian / dx**2


## Calculate residual

In [ ]:
results = {}
for run_id in run_ids:
    print('run_id:', run_id)
    flow = Flow().to(device)

    # find the files with the weights of the model and the parameters needed to initialize it
    art_path = mlflow.get_run(run_id).info.artifact_uri[8:]
    print(art_path)
    artifacts = client.list_artifacts(run_id)
    for artifact in artifacts:
        if 'model_weights.pt' in artifact.path:
            model_path = os.path.join(art_path, artifact.path)
            print(model_path)

    flow.load_state_dict(torch.load(model_path))
    print(flow)
    flow.to(device)

    # get mlflow parameter apply_gaussian_filter
    apply_gaussian_filter = (client.get_run(run_id).data.params['apply_gaussian_filter'].lower() == 'true')
    try:
        sigma = float(client.get_run(run_id).data.params['sigma'])
        print('sigma:', sigma)
    except KeyError:
        pass

    print('apply_gaussian_filter:', apply_gaussian_filter)
    flow.eval()

    results[run_id] = {}

    # Evaluate model for various nr of FM steps
    for n_steps in [4, 16, 64, 256]:
        print('n_steps:', n_steps)
        time_steps = torch.linspace(0, 1.0, n_steps + 1).to(device)

        residual_real = []
        residual_pred = []
        with torch.no_grad():
            # set random seed
            torch.manual_seed(42)

            for j, [epsmu, target] in enumerate(test_loader):
                # print(f'Batch {j}')

                cond = epsmu.to(device)

                # Initial guess
                x = torch.randn(len(cond), 1, T, N, device=device) # initial Gaussian noise
                x = torch.cumsum(x, dim=2)  # cumulative sum over time
                # print('apply_gaussian_filter:', apply_gaussian_filter)
                if apply_gaussian_filter:
                    # print('here')
                    # print('apply_gaussian_filter:', apply_gaussian_filter)
                    # print('here')
                    x = torch.as_tensor(gaussian_filter(x.cpu().numpy(), sigma=sigma, mode='wrap', axes=-1), dtype=torch.float32).to(device)  # apply Gaussian filter to make neighboring points more similar
                # mean random walk displacement for each time
                mean_disp = 1.0*torch.sqrt(2*(torch.arange(T, dtype=torch.float32, device=device)+1)/torch.pi)  # 𝜎√(2𝑁/𝜋)
                x = x / mean_disp.unsqueeze(-1) * target_std.to(device)  # scale to match target std

                for i in range(n_steps):
                    x = flow.step(x_t=x, t_start=time_steps[i], t_end=time_steps[i + 1], cond=cond)

                target = target.numpy()
                x = x[:, 0].cpu().numpy()

                # Allen-Cahn equation: du/dt = eps^2 * laplacian u - (u^3 - mu*u)

                eps = cond[:, 0].reshape(-1, 1, 1).cpu().numpy()
                mu = cond[:, 1].reshape(-1, 1, 1).cpu().numpy()

                # laplacian shape [N, N]
                # x.shape: [B, T, N]
                # cond.shape: [B,]
                lhs1 = (x[:, 1:] - x[:, :-1])/dt
                rhs1a = eps**2 * np.einsum('ij,kmj->kmi', laplacian, x[:, 1:])
                rhs1b = - (x[:, :-1]**3 - mu * x[:, :-1])
                diff1 = lhs1 - (rhs1a + rhs1b)  # shape [B, T-1, N]
                residual_pred.extend(np.linalg.norm(diff1, axis=(1,2)).tolist())

                lhs2 = (target[:, 1:] - target[:, :-1])/dt
                rhs2 = eps**2 * np.einsum('ij,kmj->kmi', laplacian, target[:, 1:]) - (target[:, :-1]**3 - mu * target[:, :-1])
                diff2 = lhs2 - rhs2  # shape [B, T-1, N]
                residual_real.extend(np.linalg.norm(diff2, axis=(1,2)).tolist())

        print('Residual (Real):', np.mean(residual_real))
        print('Residual (Predicted):', np.mean(residual_pred))

        results[run_id][n_steps] = np.mean(residual_pred)
    print('-----------')

In [ ]:
results

In [ ]:
for run_id in results:
    print('run_id:', run_id)
    SM1 = client.get_run(run_id).data.params['symmetric_matching'].lower() == 'true'
    SM3 = client.get_run(run_id).data.params['do_reflect_and_invert'].lower() == 'true'
    gf = client.get_run(run_id).data.params['apply_gaussian_filter'].lower() == 'true'

    print('symmetric_matching:', SM1)
    print('do_reflect_and_invert:', SM3)
    print('apply_gaussian_filter:', gf)

    if SM3: name = f"SM3, {'gf' if gf else 'no gf'}"
    elif SM1: name = f"SM1, {'gf' if gf else 'no gf'}"
    else: name = f"No SM, {'gf' if gf else 'no gf'}"
    print(name)

    # if gf: continue

    plt.plot(results[run_id].keys(), results[run_id].values(), label=name)

    print("-------------")

plt.legend()
plt.xlabel('Number of FM steps')
plt.ylabel('Residual')
plt.xscale('log')
plt.yscale('log')
plt.xticks([4, 16, 64, 256], [4, 16, 64, 256])

## Make images

In [ ]:
# make images of several predictions from each model

results = {}
for run_id in run_ids:
    print('run_id:', run_id)
    flow = Flow().to(device)

    # find the files with the weights of the model and the parameters needed to initialize it
    art_path = mlflow.get_run(run_id).info.artifact_uri[8:]
    # print(art_path)
    artifacts = client.list_artifacts(run_id)
    for artifact in artifacts:
        if 'model_weights.pt' in artifact.path:
            model_path = os.path.join(art_path, artifact.path)
            print(model_path)

    flow.load_state_dict(torch.load(model_path))
    # print(flow)
    flow.to(device)

    # get mlflow parameter apply_gaussian_filter
    apply_gaussian_filter = (client.get_run(run_id).data.params['apply_gaussian_filter'].lower() == 'true')
    try:
        sigma = float(client.get_run(run_id).data.params['sigma'])
        # print('sigma:', sigma)
    except KeyError:
        pass

    # print('apply_gaussian_filter:', apply_gaussian_filter)
    flow.eval()

    results[run_id] = {}

    n_steps = 64
    # print('n_steps:', n_steps)
    time_steps = torch.linspace(0, 1.0, n_steps + 1).to(device)

    SM1 = client.get_run(run_id).data.params['symmetric_matching'].lower() == 'true'
    SM3 = client.get_run(run_id).data.params['do_reflect_and_invert'].lower() == 'true'
    gf = client.get_run(run_id).data.params['apply_gaussian_filter'].lower() == 'true'

    if SM3: name = f"SM3, {'gf' if gf else 'no gf'}"
    elif SM1: name = f"SM1, {'gf' if gf else 'no gf'}"
    else: name = f"No SM, {'gf' if gf else 'no gf'}"
    print(name)

    with torch.no_grad():
        # set random seed
        torch.manual_seed(42)

        for j, [epsmu, target] in enumerate(test_loader):
            print(f'Batch {j}')

            cond = epsmu.to(device)

            # Initial guess
            x = torch.randn(len(cond), 1, T, N, device=device) # initial Gaussian noise
            x = torch.cumsum(x, dim=2)  # cumulative sum over time
            print('apply_gaussian_filter:', apply_gaussian_filter)
            if apply_gaussian_filter:
                print('here')
                print('apply_gaussian_filter:', apply_gaussian_filter)
                print('here')
                x = torch.as_tensor(gaussian_filter(x.cpu().numpy(), sigma=sigma, mode='wrap', axes=-1), dtype=torch.float32).to(device)  # apply Gaussian filter to make neighboring points more similar
            # mean random walk displacement for each time
            mean_disp = 1.0*torch.sqrt(2*(torch.arange(T, dtype=torch.float32, device=device)+1)/torch.pi)  # 𝜎√(2𝑁/𝜋)
            x = x / mean_disp.unsqueeze(-1) * target_std.to(device)  # scale to match target std

            for i in range(n_steps):
                x = flow.step(x_t=x, t_start=time_steps[i], t_end=time_steps[i + 1], cond=cond)

            target = target.numpy()
            x = x[:, 0].cpu().numpy()

            # Allen-Cahn equation: du/dt = eps^2 * laplacian u - (u^3 - mu*u)

            eps = cond[:, 0].reshape(-1, 1, 1).cpu().numpy()
            mu = cond[:, 1].reshape(-1, 1, 1).cpu().numpy()

            for i in range(16):
                if eps[i, 0, 0] > 0.1: continue
                if mu[i, 0, 0] < 0.0: continue

                fig, axes = plt.subplots(1,2,figsize=(6,2), dpi=200)
                axes[0].imshow(
                    x[i].T, aspect='auto', extent=[0, L, 0, T*subsample], origin='lower', vmin=-1, vmax=1, cmap='coolwarm'
                )
                axes[0].set_xlabel('time')
                axes[0].set_ylabel('space')
                fig.colorbar(axes[0].images[0], ax=axes[0])
                axes[0].set_title(f'Predicted')

                axes[1].imshow(
                    target[i].T, aspect='auto', extent=[0, L, 0, T*subsample], origin='lower', vmin=-1, vmax=1, cmap='coolwarm'
                )
                axes[1].set_xlabel('time')
                axes[1].set_ylabel('space')
                fig.colorbar(axes[1].images[0], ax=axes[1])
                axes[1].set_title(f'Target')

                fig.suptitle(f'{name}, eps={eps[i,0,0]:.3f}, mu={mu[i,0,0]:.3f}')
                fig.subplots_adjust(wspace=0.6, top=0.75)
                plt.show()

            if j == 1:
                break

    print('-----------')

# Bifurcation diagrams

## Vary $\mu$

In [ ]:
# load ground truth
with open('AllenCahn_periodic_varyMu.pkl', 'rb') as f:
    data = pickle.load(f)

    print(data.keys())
    solutions_gt = data['solutions']
    mu_arr_gt = data['mu_arr']

# Solution norm
solution_avg = np.mean(solutions_gt[:, -1], axis=(-1))
plt.scatter(mu_arr_gt, solution_avg, s=3, alpha=0.5)
plt.xlabel('$\mu$')
plt.ylabel('Solution norm')

In [ ]:
epsilon = 0.1



results = {}
for run_id in run_ids:
    print('run_id:', run_id)
    flow = Flow().to(device)

    # find the files with the weights of the model and the parameters needed to initialize it
    art_path = mlflow.get_run(run_id).info.artifact_uri[8:]
    # print(art_path)
    artifacts = client.list_artifacts(run_id)
    for artifact in artifacts:
        if 'model_weights.pt' in artifact.path:
            model_path = os.path.join(art_path, artifact.path)
            print(model_path)

    flow.load_state_dict(torch.load(model_path))
    # print(flow)
    flow.to(device)

    # get mlflow parameter apply_gaussian_filter
    apply_gaussian_filter = (client.get_run(run_id).data.params['apply_gaussian_filter'].lower() == 'true')
    try:
        sigma = float(client.get_run(run_id).data.params['sigma'])
        # print('sigma:', sigma)
    except KeyError:
        pass

    # print('apply_gaussian_filter:', apply_gaussian_filter)
    flow.eval()

    n_steps = 64
    # print('n_steps:', n_steps)
    time_steps = torch.linspace(0, 1.0, n_steps + 1).to(device)

    SM1 = client.get_run(run_id).data.params['symmetric_matching'].lower() == 'true'
    SM3 = client.get_run(run_id).data.params['do_reflect_and_invert'].lower() == 'true'
    gf = client.get_run(run_id).data.params['apply_gaussian_filter'].lower() == 'true'

    if SM3: name = f"SM3, {'gf' if gf else 'no gf'}"
    elif SM1: name = f"SM1, {'gf' if gf else 'no gf'}"
    else: name = f"No SM, {'gf' if gf else 'no gf'}"
    print(name)

    avg_values = []
    with torch.no_grad():
        # set random seed
        torch.manual_seed(42)
        N_samples = 500
        mu_arr = torch.linspace(-0.1, 1.0, N_samples, dtype=torch.float32).to(device)
        eps_arr = torch.tensor([epsilon]*N_samples, dtype=torch.float32).to(device)
        cond = torch.cat([eps_arr.unsqueeze(1), mu_arr.unsqueeze(1)], dim=1)

        test_loader2 = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(cond),
            batch_size=32,
            shuffle=False
        )
        for j, cond_temp in enumerate(test_loader2):

            cond_temp = cond_temp[0].to(device)

            # Initial guess
            x = torch.randn(len(cond_temp), 1, T, N, device=device) # initial Gaussian noise
            x = torch.cumsum(x, dim=2)  # cumulative sum over time
            if apply_gaussian_filter:
                x = torch.as_tensor(gaussian_filter(x.cpu().numpy(), sigma=sigma, mode='wrap', axes=-1), dtype=torch.float32).to(device)  # apply Gaussian filter to make neighboring points more similar
            # mean random walk displacement for each time
            mean_disp = 1.0*torch.sqrt(2*(torch.arange(T, dtype=torch.float32, device=device)+1)/torch.pi)  # 𝜎√(2𝑁/𝜋)
            x = x / mean_disp.unsqueeze(-1) * target_std.to(device)  # scale to match target std

            for i in range(n_steps):
                # print(f'\nStep {i}')
                # print(x.shape)
                x = flow.step(x_t=x, t_start=time_steps[i], t_end=time_steps[i + 1], cond=cond_temp)

            avg_values.extend(x[..., 0, -1, :].mean(dim=-1).cpu().numpy())

        mu_arr = torch.linspace(-0.1, 1.0, N_samples, dtype=torch.float32)

        plt.figure(figsize=(4,3), dpi=200)
        plt.scatter(mu_arr.cpu().numpy(), avg_values, label='Predicted', s=3, alpha=0.3)
        plt.scatter(mu_arr_gt, solution_avg, label='Ground truth', s=3, zorder=-1)
        plt.xlabel(r'$\mu$')
        plt.ylabel(r'Average solution value at final time')
        plt.title(f'{name}, $\epsilon={epsilon}$')
        plt.legend()
        plt.show()

In [ ]:
mu_arr = torch.linspace(-0.1, 1.0, N_samples, dtype=torch.float32)

plt.figure(figsize=(4,3), dpi=200)
plt.scatter(mu_arr.cpu().numpy(), avg_values, label='Predicted', s=3, alpha=0.3)
plt.xlabel(r'$\mu$')
plt.ylabel(r'Average solution value at final time')
plt.title(f'{name}, eps={epsilon}')
plt.show()

## Vary $\epsilon$

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

# load ground truth
with open('AllenCahn_periodic_varyEps.pkl', 'rb') as f:
    data = pickle.load(f)

    print(data.keys())
    solutions_gt = data['solutions']
    epsilon_arr_gt = data['epsilon_arr']

# Solution norm
solution_norms = np.linalg.norm(solutions_gt[:, -1], axis=(-1))
plt.scatter(epsilon_arr_gt, solution_norms, s=3, alpha=0.5)
plt.xlabel('$\epsilon$')
plt.ylabel('Solution norm')
# plt.xscale('log')

In [ ]:
epsilon = 0.1

results = {}
for run_id in run_ids:
    print('run_id:', run_id)
    flow = Flow().to(device)

    # find the files with the weights of the model and the parameters needed to initialize it
    art_path = mlflow.get_run(run_id).info.artifact_uri[8:]
    # print(art_path)
    artifacts = client.list_artifacts(run_id)
    for artifact in artifacts:
        if 'model_weights.pt' in artifact.path:
            model_path = os.path.join(art_path, artifact.path)
            print(model_path)

    flow.load_state_dict(torch.load(model_path))
    # print(flow)
    flow.to(device)

    # get mlflow parameter apply_gaussian_filter
    apply_gaussian_filter = (client.get_run(run_id).data.params['apply_gaussian_filter'].lower() == 'true')
    try:
        sigma = float(client.get_run(run_id).data.params['sigma'])
        # print('sigma:', sigma)
    except KeyError:
        pass

    # print('apply_gaussian_filter:', apply_gaussian_filter)
    flow.eval()

    n_steps = 64
    # print('n_steps:', n_steps)
    time_steps = torch.linspace(0, 1.0, n_steps + 1).to(device)

    SM1 = client.get_run(run_id).data.params['symmetric_matching'].lower() == 'true'
    SM3 = client.get_run(run_id).data.params['do_reflect_and_invert'].lower() == 'true'
    gf = client.get_run(run_id).data.params['apply_gaussian_filter'].lower() == 'true'

    if SM3: name = f"SM3, {'gf' if gf else 'no gf'}"
    elif SM1: name = f"SM1, {'gf' if gf else 'no gf'}"
    else: name = f"No SM, {'gf' if gf else 'no gf'}"
    print(name)

    sol_norm = []
    with torch.no_grad():
        # set random seed
        torch.manual_seed(42)
        N_samples = 500
        eps_arr = 10**torch.linspace(-3, -1, N_samples, dtype=torch.float32).to(device)
        mu_arr = torch.tensor([1.0]*N_samples, dtype=torch.float32).to(device)
        cond = torch.cat([eps_arr.unsqueeze(1), mu_arr.unsqueeze(1)], dim=1)

        test_loader2 = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(cond),
            batch_size=256,
            shuffle=False
        )
        for j, cond_temp in enumerate(test_loader2):

            cond_temp = cond_temp[0].to(device)

            # Initial guess
            x = torch.randn(len(cond_temp), 1, T, N, device=device) # initial Gaussian noise
            x = torch.cumsum(x, dim=2)  # cumulative sum over time
            if apply_gaussian_filter:
                x = torch.as_tensor(gaussian_filter(x.cpu().numpy(), sigma=sigma, mode='wrap', axes=-1), dtype=torch.float32).to(device)  # apply Gaussian filter to make neighboring points more similar
            # mean random walk displacement for each time
            mean_disp = 1.0*torch.sqrt(2*(torch.arange(T, dtype=torch.float32, device=device)+1)/torch.pi)  # 𝜎√(2𝑁/𝜋)
            x = x / mean_disp.unsqueeze(-1) * target_std.to(device)  # scale to match target std

            for i in range(n_steps):
                # print(f'\nStep {i}')
                # print(x.shape)
                x = flow.step(x_t=x, t_start=time_steps[i], t_end=time_steps[i + 1], cond=cond_temp)

            sol_norm.extend(torch.norm(x[..., 0, -1, :], dim=-1).cpu().numpy())

        plt.figure(figsize=(4,3), dpi=200)
        plt.scatter(eps_arr.cpu().numpy(), sol_norm, label='Predicted', s=3, alpha=0.3)
        plt.scatter(epsilon_arr_gt, solution_norms, label='Ground truth', s=3, zorder=-1)
        plt.xlabel(r'$\epsilon$')
        plt.ylabel(r'Solution norm at final time')
        plt.title(f'{name}, $\mu=1.0$')
        plt.legend()
        plt.show()


In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.scatter(eps_arr.cpu().numpy(), sol_norm, label='Predicted', s=3, alpha=0.3)
plt.scatter(epsilon_arr_gt, solution_norms, label='Ground truth', s=3, zorder=-1)
plt.xlabel(r'$\epsilon$')
plt.ylabel(r'Solution norm at final time')
plt.title(f'{name}, $\mu=1.0$')
plt.legend()
plt.show()